<h1 style='color:rgb(144, 12, 63)'; align=center><font size = 8> BLOOD GLUCOSE MANAGEMENT </font></h1>

<h2 style='color:rgb(144, 12, 63)'; align=left><font size = 6> NOTEBOOK 01-01: ETL - BLOOD GLUCOSE TRACKER</font></h2>

# REVISION HISTORY

| REV | DESCRIPTION               | DATE         |  BY   | CHECK | APPROVE  |
|:---:|:-------------------------:|:------------:|:-----:|:-----:|:--------:|
| 0   | ISSUED FOR IMPLEMENTATION | 2024-SEP-02  |  IAC  |       |          |
|     |                           |              |       |       |          |

## REVISION DESCRIPTION

* **REV 0**: ETL Scripts to create the dataset to analyze blood glucose levels.

# OVERVIEW

The **E**xtract, **T**ransform, **L**oad (ETL) process is a 3-phase process that combines, cleans and organizes raw data from a data source (or multiple data sources) into a single and consistent tabular dataset. A tabular dataset is required to perform data analytics and machine learning.

The raw data sources that can be used to create a dataset include:

*  Files (music, images, spreadsheets, or other digital files)
*  Databases
*  APIs
*  Sensors
*  Web services
*  Web scraping
*  Streaming data sources

<div class="alert alert-warning" role="alert">
  <h3 class="alert-heading">NOTE:</h3>
  <p>It is best practice that every data source have it's own notebook to describe the ETL process of creating the dataset.</p>
</div>

# OBJECTIVE

This notebook describes the ETL process to create a dataset to accomplish the following:

> Create graphs and charts to analyze blood glucose levels, including additional inputs like meal descriptions that can be correlated to appropriate meal markers.

# DATA SOURCE

The following spreadsheet is the data source used to complete the objective:

> Blood Glucose Tracker.xlsx

This spreadsheet has multiple tabs. The tabs that will be used are:
1. Blood Glucose Tracker
2. Meal Tracker

## DATA SOURCE LOCATION

A copy of this spreadsheet can be found in the following path:

`00_Data/00_Datasets/Blood Glucose Tracker.xlsx`

# REQUIRED LIBRARIES

The following Python libraries are required to perform these ETL activities:

In [ ]:
# Library to create and handle a tabular dataset
import pandas as pd

# EXTRACT

The following path and filename will be used in the scripts to extract the required data.

In [ ]:
# Path to the spreadsheet
dataPath = ".././00_Data/00_Datasets/"

In [ ]:
# Spreadsheet file name
fileName = "Blood Glucose Tracker.xlsx"

## BLOOD GLUCOSE TRACKER

The following scripts will extract data from the `Blood Glucose Tracker` tab of this spreadsheet.

In [ ]:
# The Blood Glucose Tracker tab to extract
bloodGlucoseTab = "Blood Glucose Tracker"

In [ ]:
# Extract the blood glucose data from the spreadsheet
bloodGlucose_df = pd.read_excel(
    dataPath + fileName,
    sheet_name = bloodGlucoseTab,
    dtype = {
        "Date" : str,
        "Time" : str,
        "Marker" : str,
        "Glucose (mmol/L)" : float
    }
)

# Verify that the correct data has been extracted
bloodGlucose_df

## MEAL TRACKER

The following scripts will extract data from the `Meal Tracker` tab of this spreadsheet.

In [ ]:
# The Blood Glucose Tracker tab to extract
mealTab = "Meal Tracker"

In [ ]:
# Extract the meal data from the spreadsheet
meal_df = pd.read_excel(
    dataPath + fileName,
    sheet_name = mealTab,
    dtype = {
        "Date" : str,
        "Time" : str,
        "Meal Type" : str,
        "Description" : str
    }
)

# Verify that the correct data has been extracted
meal_df

# TRANSFORM

This step involves cleaning, structuring, and converting the data into a format suitable for analysis and storage in the target system. 

## TRANSFORMATION TASKS

Some common transformation tasks include:

* [X] [~~Filtering out unnecessary data~~](#FILTERING-OUT-UNNECESSARY-DATA)
* [X] [Standardizing formats (e.g., date formats, units of measurement)](#STANDARDIZING-FORMATS)
* [X] [~~Data type conversions: Changing data types to ensure consistency~~](#DATA-TYPE-CONVERSIONS)
* [X] [Joining or merging data from different sources](#JOINING-OR-MERGING-DATA-FROM-DIFFERENT-SOURCES)
* [X] [~~Aggregations: Summarizing data or performing calculations~~](#AGGREGATIONS)
* [X] [Data cleaning: Handling missing values, correcting errors, and removing duplicates](#DATA-CLEANING)

## FILTERING OUT UNNECESSARY DATA

This step is not applicable. The extracted data does not require any filtering.

## STANDARDIZING FORMATS

### CLEANING THE DATE COLUMN

The first step in standardizing the formats is to clean up the `Date` column in both data frames by removing the `00:00:00` part of the date. This will help in any future merging of the Date and Time values.

In [ ]:
# Remove the ' 00:00:00' from the 'Date' column
bloodGlucose_df['Date'] = bloodGlucose_df['Date'].str.replace(' 00:00:00', '')

# Verify the operation
bloodGlucose_df

In [ ]:
# Remove the ' 00:00:00' from the 'Date' column
meal_df['Date'] = meal_df['Date'].str.replace(' 00:00:00', '')

# Verify the operation
meal_df

### MERGING DATE AND TIME COLUMNS

To help in graphing and the visualization of blood glucose data, a `DateTime` field will be added to **bloodGlucose_df**. Adding this field will create standardized date and time format. No other standardization is required. 

In [ ]:
# Combine the Date and Time columns into a DateTime column that will be used in graphing
bloodGlucose_df['DateTime'] = pd.to_datetime(bloodGlucose_df['Date'] + ' ' + bloodGlucose_df['Time'])

# Drop and re-order columns
bloodGlucose_df = bloodGlucose_df[
    [
        'DateTime', 
        'Date', 
        'Time', 
        'Glucose (mmol/L)', 
        'Marker'
    ]
]

# Verify results
bloodGlucose_df

## DATA TYPE CONVERSIONS

This step is not applicable. There is no need for data type conversions in these datasets.

## JOINING OR MERGING DATA FROM DIFFERENT SOURCES

In order to create a combined dataset based on information found in both data frames, there must be some common fields. To help in this merge, a new column will be added to the `meal_df` data frame. This new column will be called `Marker`.

In [ ]:
# Values that will be added to the Marker column in the meal_df
mealMarkerValues = {
    'Breakfast' : 'After Breakfast',
    'Lunch' : 'After Lunch',
    'Snack' : 'After Snack',
    'Supper' : 'After Supper'
}

In [ ]:
# Create a new column of values
meal_df['Marker'] = meal_df['Meal Type'].map(mealMarkerValues)

# Re-order columns
meal_df = meal_df[
    [
        'Date', 
        'Time', 
        'Meal Type', 
        'Marker', 
        'Description'
    ]
]

# Verify operation
meal_df

The two data frames can now be merged by `Date` and `Marker`. The following script will create a new data frame by merging `bloodGlucose_df` and `meal_df`.

In [ ]:
# Create a new data frame by merging the extracted data
bloodGlucoseManager_df = pd.merge(
    bloodGlucose_df,
    meal_df,
    how = 'left',
    on = ['Date', 'Marker'],
    suffixes = ('Test', 'Meal'), # Help determine if the time stamp is from the blood test time, or the time when the meal was eaten
)
    
# Re-order columns
bloodGlucoseManager_df = bloodGlucoseManager_df[
    [
        'DateTime', 
        'Date', 
        'TimeTest', 
        'TimeMeal', 
        'Glucose (mmol/L)',
        'Marker', 
        'Meal Type',
        'Description'
    ]
]

# Verify results
bloodGlucoseManager_df

##  AGGREGATIONS

This step is not applicable. There is no need for data type conversions in these datasets.

## DATA CLEANING

Although there are missing values in some fields, these missing values are acceptable. These missing values will not impact the analysis. The dataset does not have an errors or duplicates.

# LOAD

This step loads the newly created dataset to a target system or location. For the purpose of this project, the dataset will be loaded to the following path:

> 00_Data/00_Datasets/

The following scripts are used to load the dataset to the following target location, as a `CSV` file.

In [ ]:
# Target location of where data will be stored
targetDataPath = ".././00_Data/00_Datasets/"

In [ ]:
csvFileName = 'bloodGlucoseAnalytics.csv'

In [ ]:
bloodGlucoseManager_df.to_csv(
    targetDataPath + csvFileName,
    index = False
)

# CONCLUSION

The ETL scripts in this notebook create a dataset suitable for analytics of blood glucose levels, from the `Blood Glucose Tracker.xlsx` spreadsheet.